<a href="https://colab.research.google.com/github/uuu4/basic-QGAN-training/blob/main/basic_QGAN_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pennylane

  Using cached PennyLane-0.40.0-py3-none-any.whl.metadata (10 kB)
  Using cached rustworkx-0.16.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (10 kB)
  Using cached tomlkit-0.13.2-py3-none-any.whl.metadata (2.7 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl.metadata (9.0 kB)
  Using cached autoray-0.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached PennyLane_Lightning-0.40.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (27 kB)
  Using cached diastatic_malt-2.15.2-py3-none-any.whl.metadata (2.6 kB)
  Using cached scipy_openblas32-0.3.29.0.0-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (56 kB)
Using cached PennyLane-0.40.0-py3-none-any.whl (2.0 MB)
Using cached autoray-0.7.0-py3-none-any.whl (930 kB)
Using cached PennyLane_Lightning-0.40.0-cp311-cp311-manylinux_2_28_x86_64.whl (2.4 MB)
Using cached rustworkx-0.16.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.1 MB)
Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 k

In [ ]:
import pennylane as qml
from pennylane import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
n_qubits = 1
dev = qml.device('default.qubit',wires=n_qubits) # setting up 1 qubit circuit for generator

In [ ]:
def generator_circuit(params):
  qml.RY(params[0],wires=0) # creating superposition
  return qml.probs(wires=0) # measurement

In [ ]:
class Discriminator(nn.Module): # a classical neural network 1 to 1
  def __init__(self):
    super().__init__()
    self.fc = nn.Linear(1,1)
  def forward(self,x):
    return torch.sigmoid(self.fc(x))

true_prob = 0.75 # 0 prob
disc = Discriminator()
opt_disc = optim.Adam(disc.parameters(),lr = 0.01)
params = np.array([0.1],requires_grad=True)
params = torch.tensor([0.1], requires_grad=True)
opt_gen = optim.Adam([params],lr=0.02)

In [ ]:
for epoch in range(1000):
  #first we need to train the discriminator
  real_sample = torch.tensor([1.0]) if np.random.rand() < true_prob else torch.tensor([0.0])
  # take a sample from generator
  gen_output = generator_circuit(params)
  fake_sample = torch.tensor([1.0]) if np.random.rand() < gen_output[0][1] else torch.tensor([[0.0]])

  #calculate the loss
  disc_loss = - torch.log(disc(real_sample)) - torch.log(1-disc(fake_sample))
  opt_disc.zero_grad()
  disc_loss.backward()
  opt_disc.step()

  #train the generator
  gen_output = generator_circuit(params)
  fake_sample = torch.tensor([[1.0]]) if np.random.rand() < gen_output[0][1] else torch.tensor([[0.0]])
  gen_loss = - torch.log(disc(fake_sample))
  opt_gen.zero_grad()
  gen_loss.backward()
  opt_gen.step()

trained_probs = generator_circuit(params)
print(f"Trained Generator |0> Probabilities: {trained_probs[0][0]:.2f}")